In [94]:
# Initial setup
import pandas as pd
import numpy as np
import re # Regular expressions
dir = '/home/josh/Documents/Personal/Kaggle/NCAA'

data = [re.split('_', id) for id in sample_submission['Id']]
data = pd.DataFrame(data)
data.columns = ['Year', 'team_id1', 'team_id2']
data['Pred'] = sample_submission['Pred']
data
def make_submission(data):
    submission = pd.DataFrame(data['Year'] + '_' + data['team_id1'] +data['team_id2'])
    submission['Pred'] = data['Pred']
    submission.columns = ['Id', 'Pred']
    return(submission)
make_submission(data)

,Id,Pred
0,2012_11041124,0.5
1,2012_11041125,0.5
2,2012_11041140,0.5
3,2012_11041143,0.5
4,2012_11041153,0.5
5,2012_11041160,0.5
6,2012_11041161,0.5
7,2012_11041163,0.5
8,2012_11041166,0.5
9,2012_11041172,0.5


In [45]:
# Read in data
seasons = pd.read_csv(dir + '/Seasons.csv')
teams = pd.read_csv(dir + '/Teams.csv')
regular_compact_results = pd.read_csv(dir + '/RegularSeasonCompactResults.csv')
regular_detailed_results = pd.read_csv(dir + '/RegularSeasonDetailedResults.csv')
tourney_compact_results = pd.read_csv(dir + '/TourneyCompactResults.csv')
tourney_detailed_results = pd.read_csv(dir + '/TourneyDetailedResults.csv')
tourney_seed = pd.read_csv(dir + '/TourneySeeds.csv')
tourney_slots = pd.read_csv(dir + '/TourneySlots.csv')
sample_submission = pd.read_csv(dir + '/SampleSubmission.csv')

In [54]:
# Look at the first few rows of each dataset, and their dimensions
# sample_submission.iloc[np.arange(5)]
nrow = 4
print('seasons:\n', seasons.head(nrow), '\nDimension: ', seasons.shape)
print('teams:\n', teams.head(nrow), '\nDimension: ', teams.shape)
print('regular_compact_results:\n', regular_compact_results.head(nrow),
      '\nDimension: ', regular_compact_results.shape)
print('regular_detailed_results:\n', regular_detailed_results.head(nrow),
      '\nDimension: ', regular_detailed_results.shape)
print('tourney_compact_results:\n', tourney_compact_results.head(nrow),
      '\nDimension: ', tourney_compact_results.shape)
print('tourney_detailed_results:\n', tourney_detailed_results.head(nrow),
      '\nDimension: ', tourney_detailed_results.shape)
print('tourney_seed:\n', tourney_seed.head(nrow),
      '\nDimension: ', tourney_seed.shape)
print('tourney_slots:\n', tourney_slots.head(nrow),
      '\nDimension: ', tourney_slots.shape)
print('sample_submission:\n', sample_submission.head(nrow),
      '\nDimension: ', sample_submission.shape)

seasons:
    Season     Dayzero Regionw    Regionx    Regiony    Regionz
0    1985  10/29/1984    East       West    Midwest  Southeast
1    1986  10/28/1985    East    Midwest  Southeast       West
2    1987  10/27/1986    East  Southeast    Midwest       West
3    1988  11/02/1987    East    Midwest  Southeast       West 
Dimension:  (32, 6)
teams:
    Team_Id    Team_Name
0     1101  Abilene Chr
1     1102    Air Force
2     1103        Akron
3     1104      Alabama 
Dimension:  (364, 2)
regular_compact_results:
    Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot
0    1985      20   1228      81   1328      64    N      0
1    1985      25   1106      77   1354      70    H      0
2    1985      25   1112      63   1223      56    H      0
3    1985      25   1165      70   1432      54    H      0 
Dimension:  (139920, 8)
regular_detailed_results:
    Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot  Wfgm  Wfga ...   \
0    2003      10   1104      68   1328      6